# Bricklayer Robot

In [1]:
# Bookkeeping
# Python path for the local Drake and Manipulation
import sys
sys.path.append('/home/nikita/env/lib/python3.8/site-packages')
sys.path.append('/home/nikita/robotics/manipulation')
sys.path.append('/home/nikita/robotics')

# Cool stuff
import shutil
import os as os
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource, GenerateHtml, GetDrakePath)

In [2]:
# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

Working in directory  /home/nikita/robotics/project


In [3]:
# Hack: move our model packages to the manipulation/ folder to allow Drake to find it
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", "/home/nikita/robotics/manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", "/home/nikita/robotics/manipulation/manipulation/models/real_brick.sdf")

'/home/nikita/robotics/manipulation/manipulation/models/real_brick.sdf'

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [16]:
# iiwa robot class
# This class is based on the standard ManipulationStation system from the class.
# However, I have replaced the model with our brick environment as defined in construction.dmd.yaml
class IIWA_Painter():
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        # set up the system of manipulation station
        self.station = MakeManipulationStation(filename=FindResource(kProjectDir + "/models/construction.dmd.yaml"))

        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj is not None:
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa_position_measured"),
                self.controller.GetInputPort("iiwa_position"))

        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(wsg_position.get_output_port(),
                        self.station.GetInputPort("wsg_position"))

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext()
        self.diagram.Publish(context)
        
        # Get grip fingers
        self.left_finger = painter.station.GetSubsystemByName("plant").GetFrameByName('left_finger')
        self.right_finger = painter.station.GetSubsystemByName("plant").GetFrameByName('right_finger')

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies
        
        Input: 
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.
        
        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        # provide initial states
        q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context


    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        
    # Function to move grip fingers
    def finger_open(self):
        painter.station.GetSubsystemByName("plant").GetFrameByName('brick0_origin').CalcPoseInWorld(plant_context)

class PseudoInverseController(LeafSystem):
    """
    same controller seen in-class
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)


In [24]:
# Instantiate the robot
painter = IIWA_Painter()

# Visualize diagram
painter.diagram.set_name("diagram")
HTML('<script src="https://unpkg.com/gojs/release/go.js"></script>' + GenerateHtml(painter.diagram))

In [17]:
# Visualize the grip frame
painter.visualize_frame('gripper_current', painter.get_X_WG())

# Visualize the brick frame
context = painter.CreateDefaultContext()
plant_context = painter.plant.GetMyMutableContextFromRoot(context)
X_PF = painter.station.GetSubsystemByName("plant").GetFrameByName('brick0_origin').CalcPoseInWorld(plant_context)
painter.visualize_frame('brick_current', X_PF)

In [21]:
# Move grip to the brick
X_WG_init = painter.get_X_WG()

X_WG_target = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), np.array([-0.3, 0.3, 0.5]))

G_T = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), np.array([0, 0, 0.1]))
key_frame_poses = [X_WG_init, X_PF @ G_T, X_WG_target]

total_time = 10
times = np.linspace(0, total_time, 3)

traj = PiecewisePose.MakeLinear(times, key_frame_poses)
print(traj)

# Move
painter = IIWA_Painter(traj)
painter.paint(sim_duration=total_time)

<pydrake.trajectories.PiecewisePose_[float] object at 0x7f50c93f7330>


In [22]:
# Open grip
painter.station.GetSubsystemByName("plant").GetFrameByName

<pydrake.multibody.plant.MultibodyPlant_[float] at 0x7f50cb601230>